# A1. Graph API 取り込み - セットアップ

## 概要

このノートブックでは、Microsoft Graph API を使った取り込みの準備を行います:

1. **認証**: MSAL でアクセストークン取得
2. **サイト解決**: SharePoint サイトの ID 取得
3. **ドライブ列挙**: 対象ドキュメントライブラリの特定
4. **ファイル探索**: 取り込み対象ファイルのリストアップ

## 前提条件

- `00_prereqs_and_env.ipynb` が完了していること
- Azure AD アプリに `Sites.Read.All` または `Files.Read.All` 権限があること
- 管理者の同意が完了していること

## 1. 環境の初期化

In [ ]:
import os
import json
import msal
import requests
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List, Optional
from urllib.parse import quote

# 環境変数の読み込み
load_dotenv(Path("..") / ".env")

print("✅ ライブラリをインポートしました")

## 2. Graph API クライアントの作成

In [ ]:
class GraphAPIClient:
    """Microsoft Graph API クライアント"""
    
    def __init__(self, tenant_id: str, client_id: str, client_secret: str):
        self.tenant_id = tenant_id
        self.client_id = client_id
        self.authority = f"https://login.microsoftonline.com/{tenant_id}"
        self.base_url = "https://graph.microsoft.com/v1.0"
        
        # MSAL アプリケーション
        self.app = msal.ConfidentialClientApplication(
            client_id,
            authority=self.authority,
            client_credential=client_secret
        )
        
        self.token = None
        print(f"✅ Graph API クライアントを初期化しました (Tenant: {tenant_id[:8]}...)")
    
    def get_token(self) -> str:
        """アクセストークンを取得"""
        result = self.app.acquire_token_for_client(
            scopes=["https://graph.microsoft.com/.default"]
        )
        
        if "access_token" in result:
            self.token = result["access_token"]
            return self.token
        else:
            error = result.get("error_description", result.get("error"))
            raise Exception(f"トークン取得エラー: {error}")
    
    def get_headers(self) -> Dict[str, str]:
        """リクエストヘッダーを取得"""
        if not self.token:
            self.get_token()
        
        return {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }
    
    def get(self, endpoint: str, params: Optional[Dict] = None) -> Dict:
        """GET リクエスト"""
        url = f"{self.base_url}{endpoint}"
        response = requests.get(url, headers=self.get_headers(), params=params)
        response.raise_for_status()
        return response.json()

# クライアントの作成
graph_client = GraphAPIClient(
    tenant_id=os.getenv("GRAPH_TENANT_ID"),
    client_id=os.getenv("GRAPH_CLIENT_ID"),
    client_secret=os.getenv("GRAPH_CLIENT_SECRET")
)

# トークン取得テスト
try:
    token = graph_client.get_token()
    print(f"✅ アクセストークンを取得しました (長さ: {len(token)} 文字)")
except Exception as e:
    print(f"❌ エラー: {e}")

## 3. SharePoint サイトの解決

In [ ]:
def resolve_site_id(graph_client: GraphAPIClient, hostname: str, site_path: str) -> Dict:
    """
    SharePoint サイトの ID を取得
    
    Args:
        hostname: SharePoint ホスト名 (例: contoso.sharepoint.com)
        site_path: サイトパス (例: /sites/ai-search-handson)
    
    Returns:
        サイト情報
    """
    # サイトパスをエンコード
    encoded_path = quote(site_path, safe="")
    
    # サイト情報の取得
    endpoint = f"/sites/{hostname}:{site_path}"
    site_info = graph_client.get(endpoint)
    
    return {
        "id": site_info["id"],
        "name": site_info["displayName"],
        "webUrl": site_info["webUrl"],
        "hostname": hostname,
        "path": site_path
    }

# サイトの解決
sp_hostname = os.getenv("SP_HOSTNAME")
sp_site_path = os.getenv("SP_SITE_PATHS")

print(f"SharePoint サイトを解決しています...")
print(f"  ホスト: {sp_hostname}")
print(f"  パス: {sp_site_path}")
print()

try:
    site_info = resolve_site_id(graph_client, sp_hostname, sp_site_path)
    
    print("=" * 60)
    print("✅ サイト情報")
    print("=" * 60)
    print(f"サイト名: {site_info['name']}")
    print(f"サイトID: {site_info['id']}")
    print(f"URL: {site_info['webUrl']}")
    print("=" * 60)
    
except requests.exceptions.HTTPError as e:
    if e.response.status_code == 404:
        print(f"❌ サイトが見つかりません: {sp_hostname}{sp_site_path}")
        print("   サイトパスを確認してください")
    elif e.response.status_code == 403:
        print(f"❌ アクセス権限がありません")
        print("   アプリに Sites.Read.All 権限が付与されているか確認してください")
    else:
        print(f"❌ エラー: {e}")
    raise

## 4. ドキュメントライブラリ (ドライブ) の列挙

In [ ]:
def list_site_drives(graph_client: GraphAPIClient, site_id: str) -> List[Dict]:
    """
    サイト内のすべてのドライブ (ドキュメントライブラリ) を取得
    
    Args:
        site_id: サイトID
    
    Returns:
        ドライブのリスト
    """
    endpoint = f"/sites/{site_id}/drives"
    response = graph_client.get(endpoint)
    
    drives = []
    for drive in response.get("value", []):
        drives.append({
            "id": drive["id"],
            "name": drive["name"],
            "description": drive.get("description", ""),
            "driveType": drive.get("driveType", "documentLibrary"),
            "webUrl": drive.get("webUrl", "")
        })
    
    return drives

# ドライブの列挙
print("ドキュメントライブラリを取得しています...\n")

drives = list_site_drives(graph_client, site_info["id"])

print("=" * 60)
print(f"✅ {len(drives)} 件のドキュメントライブラリが見つかりました")
print("=" * 60)
for i, drive in enumerate(drives, 1):
    print(f"\n[{i}] {drive['name']}")
    print(f"    ID: {drive['id']}")
    print(f"    種類: {drive['driveType']}")
    if drive['description']:
        print(f"    説明: {drive['description']}")
    if drive['webUrl']:
        print(f"    URL: {drive['webUrl']}")

print("\n" + "=" * 60)

# 対象ライブラリの選択
target_library_names = os.getenv("SP_LIBRARY_NAMES", "Documents").split(",")
target_drives = [d for d in drives if d["name"] in target_library_names]

if target_drives:
    print(f"\n対象ライブラリ: {', '.join([d['name'] for d in target_drives])}")
else:
    print(f"\n⚠️  指定されたライブラリが見つかりません: {target_library_names}")
    print("   最初のライブラリを使用します")
    target_drives = drives[:1] if drives else []

## 5. ファイルの探索 (サンプル)

In [ ]:
def list_drive_items(
    graph_client: GraphAPIClient,
    drive_id: str,
    folder_path: str = "root",
    max_items: int = 50
) -> List[Dict]:
    """
    ドライブ内のアイテムを取得
    
    Args:
        drive_id: ドライブID
        folder_path: フォルダパス ("root" または "root:/path/to/folder:")
        max_items: 最大取得数
    
    Returns:
        アイテムのリスト
    """
    endpoint = f"/drives/{drive_id}/{folder_path}/children"
    params = {"$top": max_items}
    
    response = graph_client.get(endpoint, params=params)
    
    items = []
    for item in response.get("value", []):
        item_info = {
            "id": item["id"],
            "name": item["name"],
            "type": "folder" if "folder" in item else "file",
            "webUrl": item.get("webUrl", ""),
            "size": item.get("size", 0),
            "lastModified": item.get("lastModifiedDateTime", ""),
            "createdBy": item.get("createdBy", {}).get("user", {}).get("displayName", ""),
        }
        
        if item_info["type"] == "file":
            item_info["extension"] = Path(item["name"]).suffix.lower()
            item_info["mimeType"] = item.get("file", {}).get("mimeType", "")
        
        items.append(item_info)
    
    return items

# サンプルファイルの取得
if target_drives:
    sample_drive = target_drives[0]
    print(f"\nサンプルファイルを取得しています (ライブラリ: {sample_drive['name']})...\n")
    
    try:
        items = list_drive_items(graph_client, sample_drive["id"], max_items=20)
        
        files = [item for item in items if item["type"] == "file"]
        folders = [item for item in items if item["type"] == "folder"]
        
        print("=" * 60)
        print(f"✅ {len(files)} 件のファイル, {len(folders)} 件のフォルダ")
        print("=" * 60)
        
        # サポート対象の拡張子
        supported_exts = os.getenv("SUPPORTED_EXTENSIONS", ".pdf,.docx,.pptx,.xlsx,.txt").split(",")
        
        # ファイルの表示
        print("\n📄 ファイル一覧:")
        for i, file in enumerate(files[:10], 1):
            ext = file.get("extension", "")
            supported = "✅" if ext in supported_exts else "⚠️"
            size_kb = file["size"] / 1024
            print(f"  {supported} [{i}] {file['name']}")
            print(f"      サイズ: {size_kb:.1f} KB | 拡張子: {ext} | 更新: {file['lastModified'][:10]}")
        
        if len(files) > 10:
            print(f"\n  ... 他 {len(files) - 10} 件")
        
        # フォルダの表示
        if folders:
            print("\n📁 フォルダ一覧:")
            for i, folder in enumerate(folders[:5], 1):
                print(f"  [{i}] {folder['name']}")
        
        print("\n" + "=" * 60)
        
        # サポート対象ファイルの統計
        supported_files = [f for f in files if f.get("extension") in supported_exts]
        total_size = sum(f["size"] for f in supported_files) / (1024 * 1024)
        
        print(f"\n📊 サポート対象ファイル: {len(supported_files)} / {len(files)} 件")
        print(f"📊 合計サイズ: {total_size:.2f} MB")
        
        # 拡張子別の集計
        ext_count = {}
        for f in supported_files:
            ext = f.get("extension", "unknown")
            ext_count[ext] = ext_count.get(ext, 0) + 1
        
        if ext_count:
            print("\n📊 拡張子別:")
            for ext, count in sorted(ext_count.items()):
                print(f"   {ext}: {count} 件")
        
    except Exception as e:
        print(f"❌ エラー: {e}")
else:
    print("⚠️  対象ドライブがありません")

## 6. 設定の保存

In [ ]:
# 次のノートブックで使用する設定を保存
config = {
    "site_info": site_info,
    "target_drives": target_drives,
    "supported_extensions": os.getenv("SUPPORTED_EXTENSIONS", ".pdf,.docx,.pptx,.xlsx,.txt").split(","),
    "chunk_size": int(os.getenv("CHUNK_SIZE", 1000)),
    "chunk_overlap": int(os.getenv("CHUNK_OVERLAP", 200)),
    "max_files": int(os.getenv("MAX_FILES_TO_PROCESS", 100))
}

config_path = Path("..") / "config.json"
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"✅ 設定を保存しました: {config_path}")

## まとめ

このノートブックで以下を完了しました:

✅ Microsoft Graph API への認証  
✅ SharePoint サイトの解決  
✅ ドキュメントライブラリの列挙  
✅ 対象ファイルの探索  
✅ 設定の保存  

### 次のステップ

**A2_graph_ingest_run.ipynb** に進んで、実際のファイル取り込みを実行します。

### トラブルシューティング

**権限エラーが出る場合**:
- Azure AD アプリに `Sites.Read.All` または `Files.Read.All` が付与されているか確認
- 管理者の同意が完了しているか確認

**サイトが見つからない場合**:
- `SP_SITE_PATHS` の値を確認 (例: `/sites/your-site`)
- SharePoint 上でサイトが存在するか確認

**ファイルが表示されない場合**:
- `SP_LIBRARY_NAMES` の値を確認
- ライブラリにファイルがアップロードされているか確認